In [39]:
# original_data = pd.DataFrame(pd.read_csv('InorganicFormula_selected_columns.csv'))
import pandas as pd

# 读取数据
original_data = pd.DataFrame(pd.read_csv('3-原始数据集-只有化学式.csv'))

In [41]:
import pandas as pd

# 读取数据
original_data = pd.read_csv('3-原始数据集-只有化学式.csv')

# 检查每一列中空值的数量
null_values = original_data.isnull().sum()

# 输出每列的空值数量
print(null_values)


composition       0
Doping element    0
dtype: int64


In [42]:
# 打印包含空值的行和列的位置
empty_positions = original_data[original_data.isnull().any(axis=1)].index
print("空值所在的行索引:", empty_positions)

# 打印每列的空值位置
for column in original_data.columns:
    missing_data = original_data[original_data[column].isnull()]
    if not missing_data.empty:
        print(f"列 '{column}' 中空值的位置：")
        print(missing_data)


空值所在的行索引: Index([], dtype='int64')


In [43]:
import pandas as pd

# 假设 original_data 是您的原始数据集
# 创建一个空字典，用于存储新的数据集
new_datasets = {}

# 遍历原始数据集的每一列
for col_name in original_data.columns:
    # 创建新的数据集，将当前列命名为 'Name'
    new_dataset = pd.DataFrame({ 'Name': original_data[col_name] })
    
    # 将新数据集存储在字典中，字典的键是 'data1'，'data2'，依此类推
    new_datasets['data' + str(len(new_datasets) + 1)] = new_dataset

# 打印或使用新的数据集
for key, value in new_datasets.items():
    print(f"{key}:\n{value}\n")
# # 获取并使用 "data1" 数据集
# data1 = new_datasets['data3']
# print("Data1:")
# print(data1)
# # 保存 "data1" 数据集到 CSV 文件
# data1.to_csv('data1.csv', index=False)


data1:
                                   Name
0       NaNi0.330Mn0.330Fe0.330Sc0.01O2
1       NaNi0.331Mn0.331Fe0.331V0.005O2
2      Na1.00Ni0.40Fe0.10Mn0.40Ti0.10O2
3       Na0.90K0.10Ni0.40Fe0.20Mn0.40O2
4        NaNi0.25Fe0.455Al0.045Mn0.25O2
5   Na[Li0.05(Ni0.25Fe0.25Mn0.5)0.95]O2
6        NaNi0.325Mn0.33Fe0.33Zn0.005O2
7           NaNi0.33Fe0.33Mn0.33Y0.01O2
8          NaNi0.33Fe0.33Mn0.31Zr0.02O2
9    Na[Ni0.33Fe0.33Mn0.33]0.99Ta0.01O2
10          Na(Ni0.2Fe0.4Mn0.4)B0.02O2 
11      Na(Ni0.2Fe0.4Mn0.4)0.96Co0.04O2
12         NaNi0.33Fe0.33Mn0.31Sn0.02O2
13     Na0.96Ca0.02Ni0.33Fe0.33Mn0.33O2

data2:
   Name
0    Sc
1     V
2    Ti
3     K
4    Al
5    Li
6    Zn
7     Y
8    Zr
9    Ta
10    B
11   Co
12   Sn
13   Ca



In [44]:
import pandas as pd
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition.orbital import AtomicOrbitals
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition.element import ElementFraction
from pymatgen.core import Composition

# 假设 new_datasets 是包含拆分数据集的字典，如 'data1', 'data2', ...
# 每个数据集中应该有 'Name' 列

# 初始化 StrToComposition
str_to_comp = StrToComposition(target_col_id='composition')

# 初始化 AtomicOrbitals
comp_to_orbital = AtomicOrbitals()

# 初始化 ElementProperty
features_element_property = ['Number', 'MendeleevNumber', 'AtomicWeight', 'MeltingT', 
                              'Column', 'Row', 'CovalentRadius', 'Electronegativity', 
                              'NsValence', 'NpValence', 'NdValence', 'NfValence', 'NValence', 
                              'NsUnfilled', 'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled', 
                              'GSvolume_pa', 'GSbandgap', 'GSmagmom', 'SpaceGroupNumber']
stats_element_property = ['mean', 'minimum', 'maximum', 'range', 'avg_dev', 'mode']
element_property_featurizer = ElementProperty(data_source='magpie', features=features_element_property, stats=stats_element_property)

# 初始化 ElementFraction
element_fraction = ElementFraction()

# 用于存储特征转换后的数据集
result_datasets = {}

# 遍历拆分的数据集
for i, (key, dataset) in enumerate(new_datasets.items(), start=1):
    # 特征转换1: StrToComposition
    df_comp = str_to_comp.featurize_dataframe(dataset, col_id='Name')
    # df_comp.fillna(0, inplace=True)
    
    # 特征转换2: AtomicOrbitals
    orbital_features = comp_to_orbital.featurize_dataframe(df_comp, col_id='composition',ignore_errors=True)
    orbital_features = orbital_features.iloc[:, [4, 7, 8]]  # 选择感兴趣的列
    # orbital_features.fillna(0, inplace=True)
    # print(orbital_features[orbital_features.isna().any(axis=1)])
    # 特征转换3: ElementProperty
    element_property_features = element_property_featurizer.featurize_dataframe(df_comp, col_id='composition')
    element_property_features = element_property_features.iloc[:, 2:-1]  # 选择感兴趣的列
    # element_property_features.fillna(0, inplace=True)
    # 特征转换4: ElementFraction
    element_fraction_features = element_fraction.featurize_dataframe(df_comp, col_id='composition')
    element_fraction_features = element_fraction_features.iloc[:, 2:-1]  # 选择感兴趣的列
    # element_fraction_features.fillna(0, inplace=True)
    # 添加前缀
    
    prefix_orbital = f'inorganic_formula_{i}_orbital_'
    orbital_features = orbital_features.add_prefix(prefix_orbital)
    
    prefix_element_property = f'inorganic_formula_{i}_element_property_'
    element_property_features = element_property_features.add_prefix(prefix_element_property)
    
    prefix_element_fraction = f'inorganic_formula_{i}_element_fraction_'
    element_fraction_features = element_fraction_features.add_prefix(prefix_element_fraction)
    
    # 合并特征转换后的数据集
    result_datasets[key] = pd.concat([orbital_features, element_property_features, element_fraction_features], axis=1)

    
# 合并所有数据集
merged_result = pd.concat(result_datasets.values(), axis=1)

# 将合并后的结果保存为 CSV 文件
merged_result.to_csv('merged_result.csv', index=False)

# 打印或使用合并后的结果
print(merged_result)

/public/home/huangyiru/.local/lib/python3.10/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


StrToComposition:   0%|          | 0/14 [00:00<?, ?it/s]

AtomicOrbitals:   0%|          | 0/14 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/14 [00:00<?, ?it/s]

ElementFraction:   0%|          | 0/14 [00:00<?, ?it/s]

StrToComposition:   0%|          | 0/14 [00:00<?, ?it/s]

AtomicOrbitals:   0%|          | 0/14 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/14 [00:00<?, ?it/s]

ElementFraction:   0%|          | 0/14 [00:00<?, ?it/s]

    inorganic_formula_1_orbital_HOMO_energy  \
0                                 -0.295049   
1                                 -0.295049   
2                                 -0.295049   
3                                 -0.295049   
4                                 -0.295049   
5                                 -0.295049   
6                                 -0.295049   
7                                 -0.295049   
8                                 -0.295049   
9                                 -0.295049   
10                                -0.295049   
11                                -0.295049   
12                                -0.295049   
13                                -0.295049   

    inorganic_formula_1_orbital_LUMO_energy  \
0                                 -0.295049   
1                                 -0.295049   
2                                 -0.266540   
3                                 -0.295049   
4                                 -0.295049   
5           

In [45]:
print(df_comp.columns)
print(df_comp.head())
print(df_comp.index)
print(df_comp[df_comp.isna().any(axis=1)])

Index(['Name', 'composition'], dtype='object')
  Name composition
0   Sc        (Sc)
1    V         (V)
2   Ti        (Ti)
3    K         (K)
4   Al        (Al)
RangeIndex(start=0, stop=14, step=1)
Empty DataFrame
Columns: [Name, composition]
Index: []


In [46]:
# # 有机部分
# original_data2 = pd.DataFrame(pd.read_csv('OrganicSmiles_selected_columns.csv'))

In [47]:
# new_datasets2 = {}

# # 遍历原始数据集的每一列
# for col_name in original_data2.columns:
#     # 创建新的数据集，将当前列命名为 'Name'
#     new_dataset2 = pd.DataFrame({ 'Name': original_data2[col_name] })
    
#     # 将新数据集存储在字典中，字典的键是 'organic_data1'，'organic_data2'，依此类推
#     new_datasets2['organic_data' + str(len(new_datasets2) + 1)] = new_dataset2

# # 打印或使用新的数据集
# for key, value in new_datasets2.items():
#     print(f"{key}:\n{value}\n")

In [ ]:
# import pandas as pd
# import numpy as np
# from rdkit import Chem
# from rdkit.Chem import AllChem
# from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator


# # 假设 new_datasets2 是包含拆分数据集的字典，如 'organic_data1', 'organic_data2', ...
# # 每个数据集中应该有 'Name' 列

# # 用于存储 RDKit 特征化后的数据集
# rdkit_datasets = {}

# # 遍历拆分的数据集
# for key, dataset in new_datasets2.items():
#     # 特征化 RDKit
#     rdkit_features = dataset['Name'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2))
    
#     # 将 RDKit 指纹转换为 DataFrame
#     rdkit_features_df = pd.DataFrame(list(rdkit_features.apply(lambda x: np.frombuffer(x.ToBinary(), dtype=np.uint8))))

 


#     # 添加前缀
#     prefix_rdkit = f'{key}_rdkit_'
#     rdkit_features_df = rdkit_features_df.add_prefix(prefix_rdkit)
    
#     # 存储特征化后的数据集
#     rdkit_datasets[key] = rdkit_features_df

# # 合并 RDKit 特征化后的数据集
# merged_rdkit_result = pd.concat(rdkit_datasets.values(), axis=1)
# merged_rdkit_result.fillna(0, inplace=True)
# # 将合并后的结果保存为 CSV 文件
# merged_rdkit_result.to_csv('merged_rdkit_result.csv', index=False)

# # 打印或使用合并后的 RDKit 特征化结果
# print(merged_rdkit_result)

In [ ]:
# unselected_columns=pd.DataFrame(pd.read_csv('unselected_columns.csv'))

In [ ]:
# # 合并前重置索引
# merged_result.reset_index(drop=True, inplace=True)
# merged_rdkit_result.reset_index(drop=True, inplace=True)
# unselected_columns.reset_index(drop=True, inplace=True)

# # 合并三个数据集
# all_merged_data = pd.concat([merged_result, merged_rdkit_result, unselected_columns], axis=1)

# # 将合并后的结果保存为 CSV 文件
# all_merged_data.to_csv('test_train_dataset.csv', index=False)